# Import scientific python libraries

Including the seaborn library that we will be using for the first time today.

![Seaborn Logo](https://raw.githubusercontent.com/mwaskom/seaborn/master/doc/_static/logo-wide-lightbg.svg)

The Seaborn library is a powerful Python data visualization library built on top of Matplotlib, specifically designed to create visually appealing and informative statistical graphics. It simplifies the creation of complex visualizations by providing an intuitive API and a wide array of built-in themes and color palettes, making it easier to analyze and interpret data.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Visualizing distributions of data and comparing populations

Today we are going to move to dealing with a different type of data -- igneous geochemistry data. Igneous rocks are those that crystallize from cooling magma. Different magmas have different compositions associated with their origin. During class today, we will focuse on data from lava flows (this are called volcanics rocks).

There is a big database of geochemical data from rocks called Earthchem: https://www.earthchem.org

## Import a dataframe of igneous geochemistry data

Let's deal with a subset of data from Earthchem that contains geochemical data from igneous rocks. In the data folder it is in a file called `ign.csv` although it is actually tab-separated (we need to put `sep='\t'` as a parameter). It comes from here: https://github.com/brenhinkeller/StatisticalGeochemistry

In [ ]:
igneous_data = pd.read_csv('./data/ign.csv', sep='\t', dtype={4: str})

Let's look at what data are available.

In [ ]:
igneous_data.columns

Lots of different geochemical data. Note that the major elements (e.g. `FE2O3`) are given as weight percent and the the minor elements are parts per million (ppm) (e.g. `AG`).

## Filter to look at volcanics

Let's make it so that we are only dealing with the data from volcanic rocks, by filting on `TYPE`.

In [ ]:
volcanic_data = igneous_data[igneous_data['TYPE']=='VOLCANIC']
volcanic_data.head()

## Visualizing the data

A type of visualization we have used a fair amount are histograms. Let's plot up how much SiO$_2$ there is in the volcanics rocks that are in the Earthchem database.

In [ ]:
plt.hist(volcanic_data['SIO2'],bins=100)
plt.xlabel('SiO$_2$ percentage')
plt.ylabel('number of measurements')
plt.show()

Recall that instead of having the y-axis be number of values, we can have it be density using `density=True` (normalized to form a probability density, i.e., the area under the histogram will sum to 1). Given that the count itself is rather arbitary, this can be an advantageous way to plot a distribution.

In [ ]:
plt.hist(volcanic_data['SIO2'],bins=100,density=True)
plt.show()

## Kernel density estimate

One of the ways of representing the distribution of a set of datapoints is known as the 'kernel density estimate'. This is a useful way of showing the distribution of data. It places a 'kernel' (generally a normal distribution) at each data point and then sums them up.  This avoids the awkwardness of needing to chose a bin size associated with histograms, for example.

Here is an illustration of how this works.

<img src="./images/kde.png" width = 600>

> [Source: https://commons.wikimedia.org/wiki/File:Comparison_of_1D_histogram_and_KDE.png Wikimedia Creative Commons] 

There are two choices that are consequential when developing a kernel density estimate: the shape of the "kernel" and the bandwidth that sets the width of the kernel. The shape doesn't end up mattering too much, but the bandwidth very much does. There are "rules of thumb" for the bandwidth that are implemented as the defaults (and therefore often used), but these can be adjusted and it often isn't clear what the "right choice" is. A smaller bandwidth will produce a more detailed estimate with higher peaks and valleys, while a larger bandwidth will result in a smoother, less detailed curve.

### Developing a kernel density estimate with ```kdeplot```

The seaborn function ```kdeplot``` generates a kernal density estimate and then plots it. Note that seaborn provides convenience wrapper functions around other scientific python packages. It is using `matplotlib` for plotting and it is using `statsmodels`, `scipy` and `numpy` for the statistical methods. It combines these into functions that can quickly get these tasks done. The default is a gaussian kernel and Scott's rule of thumb for the bandwidth.

In [ ]:
sns.kdeplot(volcanic_data['SIO2'],shade=True)
plt.show()

We can plot a kernel density estimate curve along with a density-normalized histogram using `sns.histplot()` with a parameter of `kde=True`

In [ ]:
sns.histplot(volcanic_data['SIO2'], kde=True, stat="density")
plt.show()

## Making a bivariant histogram/scatter plot

If we want to investigate how another aspect of the chemistry of volcanic rocks relates to silica content, we can use `sns.joinplot` to make a cross-plot. Here let's look at how iron content (FeO) relates to silica content (SIO$_2$).

In [ ]:
sns.jointplot(x=volcanic_data['SIO2'], y=volcanic_data['FEO'])
plt.show()

It is pretty hard to see what is going on there. Perhaps it will be better if we change the y limit so that it is tighter on the data and make the symbols have a transparency.

In [ ]:
sns.jointplot(x=volcanic_data['SIO2'], y=volcanic_data['FEO'],alpha=0.1)
plt.ylim(0,20)
plt.show()

## Making a bivariant kernel density plot

Still pretty hard to see what is going on in the above plot, but it is an improvement. Let's put kernel density estimates to use. Here the same kernel density estimate is used and shown for the univariate data on each axis. However, we now have a bivariate kernel density estimate as well. Taken together we can see that there is a strong relationship between SiO$_2$ levels and FeO levels in volcanic rocks.

In [ ]:
sns.jointplot(x=volcanic_data['SIO2'], 
              y=volcanic_data['FEO'],
              kind='kde',
              joint_kws={'fill': True})
plt.ylim(0,20)
plt.show()

## Mafic vs. felsic

This illustration provides an overview of the compositional difference between different types of igneous rocks:

<img src="./images/mafic_felsic.jpg" width = 600>

These compositional differences manifest in different material properties which can be observed at active volcanos:

- mafic lava flowing (Kilauea, Hawaii): https://www.youtube.com/watch?v=amTENlOiVFU
- dacite dome growing within Mount Saint Helens: https://www.youtube.com/watch?v=h6B1myUKAS4 https://www.youtube.com/watch?v=3v7p9mMV6M8 *composition of 65 percent SiO2 (dacite)*
- eruption of Mount Saint Helens: https://www.youtube.com/watch?v=AYla6q3is6w
- intermediate eruption (Krakatau, Indonesia): https://www.youtube.com/watch?v=NLhjNzQHphQ

## Evaluating compositional differences between mafic, intermediate and felsic volcanics (iron)

Let's first focus on the difference between the iron contents.

We can plot up histograms. Using the `histtype='step'` parameter enables us to see multiple histograms plotted atop one another.

In [ ]:
plt.hist(volcanic_data[volcanic_data['COMPOSITION']=='INTERMEDIATE']['FEO'],
         bins=100,density=True,label='intermediate volcanics',histtype='step')
plt.hist(volcanic_data[volcanic_data['COMPOSITION']=='FELSIC']['FEO'],
         bins=100,density=True,label='felsic volcanics',histtype='step')
plt.hist(volcanic_data[volcanic_data['COMPOSITION']=='MAFIC']['FEO'],
         bins=100,density=True,label='mafic volcanics',histtype='step')
plt.xlim(0,20)
plt.xlabel('FeO (%)')
plt.ylabel('density')
plt.legend()
plt.show()

The curves can also be plotted atop one another using `sns.kdeplot`

In [ ]:
sns.kdeplot(volcanic_data[volcanic_data['COMPOSITION']=='INTERMEDIATE']['FEO'],label='intermediate volcanics')
sns.kdeplot(volcanic_data[volcanic_data['COMPOSITION']=='FELSIC']['FEO'],label='felsic volcanics')
sns.kdeplot(volcanic_data[volcanic_data['COMPOSITION']=='MAFIC']['FEO'],label='mafic volcanics')
plt.xlim(0,20)
plt.xlabel('FeO (%)')
plt.ylabel('density')
plt.legend()
plt.show()

## Other way to visualize the distributions

### The box plot

> A box plot (or box-and-whisker plot) shows the distribution of quantitative
data in a way that facilitates comparisons between variables or across
levels of a categorical variable. The box shows the quartiles of the
dataset while the whiskers extend to show the rest of the distribution,
except for points that are determined to be "outliers" using a method
that is a function of the inter-quartile range. *From seaborn docstring*

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x="COMPOSITION", y="FEO", data=volcanic_data, order=['FELSIC','INTERMEDIATE','MAFIC'])
plt.ylim(0,20)
plt.show()

### Violin plot

Perhaps you like box plots, but you also have a new-found love of probability density estimates. Well you are in luck as the violin plot puts them both together. The width of the "violin" represents the kernel density estimate while the black bar is the same values as the box plot with the white dot being the median.

In [ ]:
plt.figure(figsize=(6,4))
sns.violinplot(x="COMPOSITION", y="FEO", data=volcanic_data, order=['FELSIC','INTERMEDIATE','MAFIC'])
plt.ylim(0,20)
plt.show()

## Evaluating compositional differences between mafic, intermediate and felsic volcanics (sodium)

Looking at the these illustration of composition plot we see that it indicates that there is more iron in mafic rocks (which looks to be the case). It also indicated that there is more sodium (Na) in felsic rocks.

<img src="./images/mafic_felsic.jpg" width = 600>

**Code for you to write**

 Make the following plots to evaluate this assertion:

- jointplot of NA2O vs SIO2 for the entire volcanic_data set. make one joint plot that is `kind='kde'` and one that is `kind='hex'`
- violin plot (`sns.violinplot`) of NA2O categorized by composition

**Code for you to write**

Once you have made these visualizations, calculate the mean NA2O in felsic, intermediate and mafic volcanics. Assign these values to be `mafic_NA20_mean`, `intermediate_NA20_mean` and `felsic_NA20_mean` and print them.

Do these values fit with the schematic illustration of the felsic/mafic diagram? What is consistent and what isn't?

*write your answer here*

# A/B Testing 

> In modern data analytics, deciding whether two numerical samples come from the same underlying distribution is called A/B testing. The name refers to the labels of the two samples, A and B. 

> **The Hypotheses**

> We can try to answer this question by a test of hypotheses. The chance model that we will test says that there is no underlying difference in the populations; the distributions in the samples are different just due to chance.

> Formally, this is the null hypothesis. We are going to have to figure out how to simulate a useful statistic under this hypothesis. But as a start, let's just state the two natural hypotheses.

*from https://www.inferentialthinking.com/chapters/12/1/AB_Testing.html*

In the example in Data 8, students are asked to consider the birth weight of babies when the mother was a smoker and when the mother was a nonsmoker. The two hypotheses and the test statistic were:

> **Null hypothesis**: In the population, the distribution of birth weights of babies is the same for mothers who don't smoke as for mothers who do. The difference in the sample is due to chance.

> **Alternative hypothesis**: In the population, the babies of the mothers who smoke have a lower birth weight, on average, than the babies of the non-smokers.

> **Test Statistic**

> The alternative hypothesis compares the average birth weights of the two groups and says that the average for the mothers who smoke is smaller. Therefore it is reasonable for us to use the difference between the two group means as our statistic.

> We will do the subtraction in the order "average weight of the smoking group  − average weight of the non-smoking group". Small values (that is, large negative values) of this statistic will favor the alternative hypothesis.

We can use this same approach in considering the sodium content of the lava samples. where the **Null hypothesis** is that sodium content is the same for rocks of different compositions and differences in the sample is the result of chance. The **Alternative hypothesis** is that the sodium content of felsic lavas is, on average, higher.

## Testing the null hypothesis that there is no difference in the sodium content of mafic and felsic lavas

To start with, let's consider the difference between mafic and felsic lavas.

We can make a simplified Dataframe that just contains `composition`, `'SIO2'`, and `'NA2O'`

In [ ]:
#filter dataframe to just be Felsic and Mafic volcanics
volcanic_felsic_mafic = volcanic_data[(volcanic_data['COMPOSITION']=='FELSIC') | (volcanic_data['COMPOSITION']=='MAFIC')]
#filter dataframe to just have composition, 'SIO2', and 'NA2O' columns
volcanic_felsic_mafic = pd.DataFrame(volcanic_felsic_mafic,columns=['COMPOSITION','SIO2','NA2O'])
#drop rows where there are missing data
volcanic_felsic_mafic = volcanic_felsic_mafic.dropna()
volcanic_felsic_mafic.tail()

We can calculate the mean of Na$_2$O in felsic and mafic rocks and calculate the difference.

In [ ]:
felsic_NA20_mean = np.mean(volcanic_felsic_mafic[volcanic_felsic_mafic['COMPOSITION']=='FELSIC']['NA2O'])
mafic_NA20_mean = np.mean(volcanic_felsic_mafic[volcanic_felsic_mafic['COMPOSITION']=='MAFIC']['NA2O'])

felsic_NA20_mean - mafic_NA20_mean

Let's define a function that can make this calculation and make a function for median difference too while we are at it:

In [ ]:
def difference_of_means(dataframe, value, category_column, category_values):
    """
    Calculate the difference between the means of two categories in a given dataframe.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The input dataframe containing the data.
    value : str
        The column name for which the mean will be calculated.
    category_column : str
        The column name that will be used to categorize the data.
    category_values : list of str
        A list containing two distinct values of the category_column used to separate the data into two groups.
        
    Returns
    -------
    float
        The difference between the means of the two categories.
    """
    mean_1 = np.mean(dataframe[dataframe[category_column] == category_values[0]][value])
    mean_2 = np.mean(dataframe[dataframe[category_column] == category_values[1]][value])
    return mean_1 - mean_2

def difference_of_medians(dataframe, value, category_column, category_values):
    """
    Calculate the difference between the medians of two categories in a given dataframe.
    
    Parameters
    ----------
    dataframe : pandas.DataFrame
        The input dataframe containing the data.
    value : str
        The column name for which the median will be calculated.
    category_column : str
        The column name that will be used to categorize the data.
    category_values : list of str
        A list containing two distinct values of the category_column used to separate the data into two groups.
        
    Returns
    -------
    float
        The difference between the medians of the two categories.
    """
    median_1 = np.median(dataframe[dataframe[category_column] == category_values[0]][value])
    median_2 = np.median(dataframe[dataframe[category_column] == category_values[1]][value])
    return median_1 - median_2

In [ ]:
felsic_mafic_mean_diff = difference_of_means(volcanic_felsic_mafic,'NA2O','COMPOSITION',['FELSIC','MAFIC'])
felsic_mafic_mean_diff

In [ ]:
felsic_mafic_median_diff = difference_of_medians(volcanic_felsic_mafic,'NA2O','COMPOSITION',['FELSIC','MAFIC'])
felsic_mafic_median_diff

## Testing the null hypothesis through random permutation

The difference of the means suggests that the alternative hypothesis that felsic lavas have more sodium on average is true. However, the difference is quite small (less than 1%). How can we test that this is not the result of chance?

> **Predicting the Statistic Under the Null Hypothesis**

> To see how the statistic should vary under the null hypothesis, we have to figure out how to simulate the statistic under that hypothesis. A clever method based on random permutations does just that

If there were no difference between the two distributions in the underlying population, then the sodium content of a lava sample should not depend on whether it is mafic or felsic. Consequently, the average should remain the same, regardless of the rock type. The idea, then, is to shuffle all the labels randomly among the rock samples. This process is called random permutation.

In [ ]:
volcanic_felsic_mafic['COMPOSITION_Shuffled'] = np.random.permutation(volcanic_felsic_mafic['COMPOSITION'].values)
volcanic_felsic_mafic.tail()

Now we can calculate the difference of the means with the shuffled labels.

In [ ]:
difference_of_means(volcanic_felsic_mafic,'NA2O','COMPOSITION_Shuffled',['FELSIC','MAFIC'])

The above difference is the result of one random permutation. We want to do many random permutations to test the hypothesis.

We will do that using the following recipe:

- Initialize two empty lists, `shuffled_fel_maf_mean` and `shuffled_fel_maf_median`, to store the differences in means and medians, respectively, for each random permutation.
- Set the number of repetitions to 1000. This is the number of times the random permutation will be performed to generate a distribution of differences under the null hypothesis.
- Create a loop that iterates over the range of repetitions (1000 times):
    - a. Shuffle the 'COMPOSITION' column of the volcanic_felsic_mafic DataFrame and store the permuted values in a new column called 'COMPOSITION_Shuffled'. This step simulates the null hypothesis by randomly assigning the 'FELSIC' and 'MAFIC' labels to the samples.
    - b. Calculate the difference in means for the shuffled data using the difference_of_means function with the 'NA2O' column as the value, 'COMPOSITION_Shuffled' as the category column, and ['FELSIC', 'MAFIC'] as the category values. Append the result to the dif_permuations_fel_maf_mean list.
    - c. Calculate the difference in medians for the shuffled data using the difference_of_medians function with the same parameters as the previous step. Append the result to the dif_permuations_fel_maf_median list.

After running this code, the shuffled_fel_maf_mean and shuffled_fel_maf_median lists will each contain 1000 values, representing the distribution of differences in means and medians, respectively, under the null hypothesis.

In [ ]:
shuffled_fel_maf_mean = []
shuffled_fel_maf_median = []

repetitions = 1000

for i in np.arange(repetitions):
    volcanic_felsic_mafic['COMPOSITION_Shuffled'] = np.random.permutation(volcanic_felsic_mafic['COMPOSITION'].values)
    
    new_mean_diff = difference_of_means(volcanic_felsic_mafic,'NA2O','COMPOSITION_Shuffled',['FELSIC','MAFIC'])
    shuffled_fel_maf_mean.append(new_mean_diff) 
    
    new_median_diff = difference_of_medians(volcanic_felsic_mafic,'NA2O','COMPOSITION_Shuffled',['FELSIC','MAFIC'])
    shuffled_fel_maf_median.append(new_median_diff) 

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.hist(shuffled_fel_maf_mean,label='permutations')
plt.xlabel('Na$_2$O mean difference')

plt.subplot(1,2,2)
plt.hist(shuffled_fel_maf_median,label='permutations')
plt.xlabel('Na$_2$O median difference')

plt.show()

Notice how the distributions are centered around 0. This makes sense, because under the null hypothesis the two groups should have roughly the same average. Therefore the difference between the group averages should be around 0.

## Comparing the random permutations to the actual difference

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.hist(shuffled_fel_maf_mean,label='permutations')
plt.scatter(felsic_mafic_mean_diff,0,color='red',zorder=1000,label='actual felsic-mafic')
plt.xlabel('Na$_2$O mean difference')
plt.legend()

plt.subplot(1,2,2)
plt.hist(shuffled_fel_maf_median,label='permutations')
plt.scatter(felsic_mafic_median_diff,0,color='red',zorder=1000,label='actual felsic-mafic')
plt.xlabel('Na$_2$O median difference')
plt.legend()
plt.show()

We can calculate the empirical p-value as a measure of the evidence against the null hypothesis. A smaller p-value indicates that the observed mean difference is less likely to have occurred by chance, whereas a larger p-value suggests that the observed difference might be due to random variation.

To calculate this, we can compare the actual mean difference to the difference that arose through the reshuffled labels. And then can do the same for the median.

In [ ]:
shuffled_fel_maf_mean_array = np.array(shuffled_fel_maf_mean)

mean_empirical_p = np.count_nonzero(shuffled_fel_maf_mean_array >= felsic_mafic_mean_diff) / repetitions
mean_empirical_p

In [ ]:
shuffled_fel_maf_median_array = np.array(shuffled_fel_maf_median)

median_empirical_p = np.count_nonzero(shuffled_fel_maf_median_array >= felsic_mafic_median_diff) / repetitions
median_empirical_p

The empirical P-values are 0, meaning that none of the 1,000 permuted samples resulted in a difference that was as large as that which is observed in the actual samples. This is only an approximation. The exact chance of getting a difference in that range is not 0, but it is vanishingly small.

So it appears to be well-supported that felsic magmas have more Na$_2$O than mafic magmas.

## Testing the null hypothesis that there is no difference in the sodium content of intermediate and felsic lavas

In [ ]:
volcanic_felsic_int = volcanic_data[(volcanic_data['COMPOSITION']=='FELSIC') | (volcanic_data['COMPOSITION']=='INTERMEDIATE')]
volcanic_felsic_int = volcanic_felsic_int[(volcanic_felsic_int['SIO2'] < 64) & (volcanic_felsic_int['SIO2'] > 51)]
volcanic_felsic_int = pd.DataFrame(volcanic_felsic_int,columns=['COMPOSITION','SIO2','NA2O'])
volcanic_felsic_int = volcanic_felsic_int.dropna()
volcanic_felsic_int.tail()

**Code for you to write**

**Calculate the difference in the median and the difference in the mean between the NA2O of intermediate and felsic lavas. Assign them to variables and print them.**

In [ ]:
felsic_int_mean_diff = #complete the code

In [ ]:
felsic_int_median_diff = #complete the code

**Code for you to write**

**Shuffle the compositions within the `volcanic_felsic_int` dataframe. Do it 1000 times and make histograms of the mean and median of these permutations that includes the actual median and mean that you calculated**

In [ ]:
shuffled_fel_int_mean = []
shuffled_fel_int_median = []

repetitions = 1000

for i in np.arange(repetitions):
    #complete the code

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
#complete the code

**Calculate the p-value for the median and mean**

Given that the mean and median of the difference is negative (meaning that there is more $Na_2O$ in the intermediate rocks, we will look at the p-value associated with that difference arising through chance (i.e. felsic intermediate difference being a larger negative difference in the shuffled data vs. the actual data).

In [ ]:
shuffled_fel_int_mean_array = np.array(shuffled_fel_int_mean)

empirical_p_mean = np.count_nonzero(shuffled_fel_int_mean_array <= felsic_int_mean_diff) / repetitions
empirical_p_mean

In [ ]:
shuffled_fel_int_mean_array = np.array(shuffled_fel_int_mean)

empirical_p_median = np.count_nonzero(shuffled_fel_int_median <= felsic_int_median_diff) / repetitions
empirical_p_median